# Preparación de conjuntos de datos para consultas

In [79]:
import re
import ast
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import nltk
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta

In [95]:
games = pd.read_parquet('data/01-steam_games.parquet')
reviews= pd.read_parquet('data/02-user_reviews.parquet')
items = pd.read_parquet('data/03-users_items.parquet')


In [96]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57367 entries, 0 to 57366
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   user_id            57367 non-null  object
 1   user_url           57367 non-null  object
 2   reviews_item_id    57367 non-null  int64 
 3   reviews_helpful    57367 non-null  object
 4   reviews_recommend  57367 non-null  bool  
 5   reviews_review     57367 non-null  object
 6   posted_year        57367 non-null  int64 
dtypes: bool(1), int64(2), object(4)
memory usage: 2.7+ MB


Se pide crear una nueva columna llamada 'sentiment_analysis' que reemplace a 'reviews_review' donde se realice un análisis de sentimiento de los comentarios con la siguiente escala:

* 0 si es malo,
* 1 si es neutral o esta sin review
* 2 si es positivo.

Dado que el objetivo de este proyecto es realizar una prueba de concepto, consiguiendo un producto mínimo viable, se realiza un análisis de sentimiento básico utilizando TextBlob que es una biblioteca de procesamiento de lenguaje natural (NLP) en Python. El objetivo de esta metodología es asignar un valor numérico a un texto, en este caso a los comentarios que los usuarios dejaron para un juego determinado, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo. 

Esta metodología toma una revisión de texto como entrada, utiliza TextBlob para calcular la polaridad de sentimiento y luego clasifica la revisión como negativa, neutral o positiva en función de la polaridad calculada. En este caso, se consideraron las polaridades por defecto del modelo, el cuál utiliza umbrales -0.2 y 0.2, siendo polaridades negativas por debajo de -0.2, positivas por encima de 0.2 y neutrales entre medio de ambos.

In [105]:
def analisis_sentimiento(review):
    '''
    Realiza un análisis de sentimiento en un texto dado y devuelve un valor numérico que representa el sentimiento.

    Esta función utiliza la librería TextBlob para analizar el sentimiento en un texto dado y
    asigna un valor numérico de acuerdo a la polaridad del sentimiento.

    Parameters:
        review (str): El texto que se va a analizar para determinar su sentimiento.

    Returns:
        int: Un valor numérico que representa el sentimiento del texto:
             - 0 para sentimiento negativo.
             - 1 para sentimiento neutral o no clasificable.
             - 2 para sentimiento positivo.
    '''
    if review is None:
        return 1
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity < -0.2:
        return 0  
    elif polarity > 0.2: 
        return 2 
    else:
        return 1 

In [107]:
reviews['sentiment_analysis'] = reviews['reviews_review'].apply(analisis_sentimiento)
reviews.head()

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,posted_year,sentiment_analysis_tb,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,1,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,1,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,2013,2,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013,2,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014,1,1


In [108]:
conteo_sentimientos = reviews['sentiment_analysis'].value_counts()
conteo_sentimientos

sentiment_analysis
1    35243
2    17097
0     5027
Name: count, dtype: int64

In [109]:
def revision_sentimiento(reviews, sentiments):
    import random
    for sentiment_value in range(3):
            print(f"Para la categoría de análisis de sentimiento {sentiment_value} se tienen estos ejemplos de reviews:")
            sentiment_reviews = [review for review, sentiment in zip(reviews, sentiments) if sentiment == sentiment_value]
            
            for i, review in enumerate(random.sample(sentiment_reviews, 5), start=1):
                print(f"Review {i}: {review}")
            
            print("="*100)

In [110]:
revision_sentimiento(reviews['reviews_review'], reviews['sentiment_analysis'])

Para la categoría de análisis de sentimiento 0 se tienen estos ejemplos de reviews:
Review 1: who dosent anyone want this game i like it please bye it
Review 2: How to save game
Review 3: All i can say is that this games matchmaking is so poor it ridiculous, to the point were every game a rank 1 hunter would always be lobbed against me and my frineds, who are not even past 20. This game is broken.
Review 4: this is an amazing game and the only disbenifits from owning this game is the glitches and how you have to spend money on everything encluding the dlc.I recomend this game to anyone that likes adventuring, killing things, absoving a dragons soul, pickpocketing and casting weird dangerous spells.
Review 5: Oh, where do I start with this game. If I could say one word for this game it would probably be.... Amazeballs! The way you feel while playing this FPS teambased game is so different from any other. This game was one of my first paid games and I will never regret it, it has made me

In [113]:
reviews = reviews.drop(columns=['reviews_review'])
reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'posted_year', 'sentiment_analysis'],
      dtype='object')

## Cantidad de dinero gastado por usuario y cantidad de items consumidos

Se crea el dataframe 'df_gastos_items' que contiene por cada usuario el gasto en videojuegos y la cantidad de items consumidos. Para ello se realizan una serie de pasos creando algunos dataframe auxiliares para luego unir los resultados.

In [134]:
items = pd.read_parquet('data/03-users_items.parquet')

In [135]:
# Se extraen las columnas 'items_count', 'user_id' e 'item_id'
gasto_items = items[['items_count', 'user_id', 'item_id']]
gasto_items.head()

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,30
2,277,76561197970982479,300
3,277,76561197970982479,240
4,277,76561197970982479,3830


Se reserva el dataframe anterior y se realiza un nuevo dataframe auxiliar, pero con el precio de cada juego.

In [136]:
precio_juego = games[['price', 'id']]
# Elimina los duplicados
precio_juego = precio_juego.drop_duplicates(subset='id', keep='first')
# Se renombra 'id' por 'item_id' para unir mas adelante
precio_juego = precio_juego.rename(columns={'id':'item_id'})
precio_juego

,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,2.99,773570
...,...,...
77913,1.99,773640
77917,4.99,733530
77920,1.99,610660
77923,4.99,658870


Finalmente, se unen los dos dataframe auxiliares para conformar un dataframe final llamado `df_gastos_items`.

In [137]:
gasto_items = gasto_items.merge(precio_juego, on='item_id', how='left')
gasto_items

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,30,4.99
2,277,76561197970982479,300,9.99
3,277,76561197970982479,240,19.99
4,277,76561197970982479,3830,9.99
...,...,...,...,...
3246370,7,76561198329548331,304930,0.00
3246371,7,76561198329548331,227940,0.00
3246372,7,76561198329548331,388490,0.00
3246373,7,76561198329548331,521570,0.00


In [138]:
gasto_items.isna().sum().sort_values(ascending= False)/len(items) * 100

price          12.738516
items_count     0.000000
user_id         0.000000
item_id         0.000000
dtype: float64

In [139]:
fill_precio = gasto_items['price'].fillna(0.0)
# Se borra la columna original y se concatena la columna rellena con todo el dataframe
gasto_items = pd.concat([gasto_items.drop('price', axis=1), fill_precio], axis=1)
gasto_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3246375 entries, 0 to 3246374
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   items_count  int64  
 1   user_id      object 
 2   item_id      int64  
 3   price        float64
dtypes: float64(1), int64(2), object(1)
memory usage: 99.1+ MB


In [142]:
gasto_items.isna().sum().sort_values(ascending= False)/len(items) * 100

items_count    0.0
user_id        0.0
item_id        0.0
price          0.0
dtype: float64

In [143]:
# Se elimina 'item_id'
gasto_items = gasto_items.drop('item_id', axis=1)

# Se agrupa por usuario sumando los precios de los juegos consumidos
gasto_items_usuario = gasto_items.groupby('user_id')['price'].sum().reset_index()
gasto_items_usuario

,user_id,price
0,--000--,187.83
1,--ace--,122.89
2,--ionex--,109.92
3,-2SV-vuLB-Kg,244.68
4,-404PageNotFound-,1159.46
...,...,...
68398,zzonci,0.00
68399,zzoptimuszz,4.99
68400,zzydrax,99.94
68401,zzyfo,484.73


Para volver a agregar los items consumidos por cada usuario se hace un dataframe auxiliar 'df_count_items' conteniendo la cantidad de items por usuario.

In [144]:
conteo_items_usuario = gasto_items[['items_count', 'user_id']]

# se eliminan los duplicados
conteo_items_usuario = conteo_items_usuario.drop_duplicates(subset='user_id', keep='first')
conteo_items_usuario

,items_count,user_id
0,277,76561197970982479
198,888,js41637
717,137,evcentric
821,328,Riot-Punch
951,541,doctr
...,...,...
3246360,5,76561198320038728
3246365,321,76561198320136420
3246367,4,ArkPlays7
3246369,22,76561198323066619


In [145]:
gasto_items = conteo_items_usuario.merge(gasto_items_usuario, on='user_id', how='right')
gasto_items

,items_count,user_id,price
0,58,--000--,187.83
1,44,--ace--,122.89
2,23,--ionex--,109.92
3,68,-2SV-vuLB-Kg,244.68
4,149,-404PageNotFound-,1159.46
...,...,...,...
68398,5,zzonci,0.00
68399,61,zzoptimuszz,4.99
68400,13,zzydrax,99.94
68401,84,zzyfo,484.73


## Playtime_forever por usuario

Se busca generar un dataframe que contenga por género de videojuego, el tiempo jugado por cada usuario con su id y url del perfil. Para ello, se realizarán algunos pasos generando algunos dataframes auxiliares.  
En primer lugar, se extrae de `df_items` las columnas 'playtime_forever', 'user_id' y 'item_id' y se reserva.

In [148]:
items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'steam_id', 'items_count',
       'user_id'],
      dtype='object')

In [149]:
tiempo_total_usuario_item = items[['playtime_forever', 'user_id', 'item_id']]
tiempo_total_usuario_item

,playtime_forever,user_id,item_id
0,0.10,76561197970982479,10
1,0.12,76561197970982479,30
2,78.88,76561197970982479,300
3,30.88,76561197970982479,240
4,5.55,76561197970982479,3830
...,...,...,...
3246370,11.28,76561198329548331,304930
3246371,0.72,76561198329548331,227940
3246372,0.05,76561198329548331,388490
3246373,0.07,76561198329548331,521570


Del dataframe `df_games` se extrae el 'id' del item y el género.

In [150]:
item_genero = games[['genres', 'id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
item_genero = item_genero.rename(columns={'id':'item_id'})
item_genero

,genres,item_id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
77925,Early Access,681550
77926,Adventure,681550
77927,Indie,681550
77928,Action,681550


Ahora se unen las dos tablas anteriores para obtener los generos de todos los usuarios con sus id y url.

In [151]:
tiempo_juego_genero = tiempo_total_usuario_item.merge(item_genero, on='item_id')
tiempo_juego_genero

,playtime_forever,user_id,item_id,genres
0,0.10,76561197970982479,10,Action
1,0.12,76561197970982479,30,Action
2,78.88,76561197970982479,300,Action
3,30.88,76561197970982479,240,Action
4,5.55,76561197970982479,3830,Action
...,...,...,...,...
7109339,0.05,76561198329548331,388490,Adventure
7109340,0.05,76561198329548331,388490,Free to Play
7109341,0.07,76561198329548331,521570,Casual
7109342,0.07,76561198329548331,521570,Free to Play


In [153]:
# Se agrupa por usuario y se suma el tiempo de juego
genero_usuario_agrupado = tiempo_juego_genero.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()

genero_usuario_agrupado

,genres,user_id,playtime_forever
0,Action,--000--,2331.40
1,Action,--ace--,1155.41
2,Action,--ionex--,639.67
3,Action,-2SV-vuLB-Kg,714.31
4,Action,-404PageNotFound-,1978.31
...,...,...,...
638370,Web Publishing,zepavil,632.10
638371,Web Publishing,zeshirky,0.02
638372,Web Publishing,zevlupine,0.07
638373,Web Publishing,zilaman,0.15


## Ranking de géneros por tiempo de juego

En este punto lo que se busca es un dataframe auxiliar que contenga el ranking de los géneros de videojuegos en función del tiempo jugados de cada uno. Se aprovecha que ya se hizo para el punto anterior un dataframe auxiliar que contiene los géneros y el tiempo jugado, por lo que se usa el mismo. Primero se agrupa por género, luego se ordena y se agrega una columna de ranking.

In [155]:
ranking_genero = genero_usuario_agrupado.groupby('genres')['playtime_forever'].sum().reset_index()
# Se ordena por 'playtime_horas'

ranking_genero = ranking_genero.sort_values(by='playtime_forever', ascending=False)

# Agregar una columna con la posición del ranking
ranking_genero['ranking'] = ranking_genero['playtime_forever'].rank(ascending=False).astype(int)
ranking_genero

,genres,playtime_forever,ranking
0,Action,53835472.34,1
9,Indie,24735585.96,2
12,RPG,17268422.89,3
14,Simulation,16813377.29,4
1,Adventure,16804379.36,5
17,Strategy,13339374.02,6
8,Free to Play,10791684.65,7
10,Massively Multiplayer,8161316.25,8
4,Casual,4215686.21,9
6,Early Access,2650326.46,10


## Cantidad de items y contenido free por desarrollador

En este punto se busca un dataframe que contenga para cada desarrollador de juegos, los items que desarrolla, el año de lanzamiento y el precio de cada uno.

In [157]:
games.columns

Index(['genres', 'price', 'early_access', 'id', 'release_year', 'publisher',
       'app_name', 'title', 'developer'],
      dtype='object')

In [158]:
precio_anual_developer = games[['price', 'release_year', 'developer', 'id']]

# Se renombra la columna 'id' para unirla con el dataframe anterior
precio_anual_developer = precio_anual_developer.rename(columns={'id':'item_id'})

# se eliminan los duplicados
items_developer = precio_anual_developer.drop_duplicates()
items_developer

,price,release_year,developer,item_id
0,4.99,2018.0,Kotoshiro,761140
5,0.00,2018.0,Secret Level SRL,643980
9,0.00,2017.0,Poolians.com,670290
14,0.99,2017.0,彼岸领域,767400
17,2.99,Dato no disponible,Sin dato disponible,773570
...,...,...,...,...
77913,1.99,2018.0,"Nikita ""Ghost_RUS""",773640
77917,4.99,2018.0,Sacada,733530
77920,1.99,2018.0,Laush Dmitriy Sergeevich,610660
77923,4.99,2017.0,"xropi,stev3ns",658870


## 'release_anio' en `df_reviews`

En este punto se agrega el año de lanzamiento de un juego al dataframe `df_reviews`. En primer lugar se extraen las columnas de id del juego y el año de lanzamiento del mismo, se borran los duplicados y luego se unen con el dataframe de reviews.

In [160]:
lanzamiento_item = games[['id', 'release_year']]

# Se renombra la columna 'id' para unirla con el dataframe anterior
lanzamiento_item = lanzamiento_item.rename(columns={'id':'reviews_item_id'})

# se eliminan los duplicados
lanzamiento_item = lanzamiento_item.drop_duplicates()

# Se une el lanzamiento con el item de reviews
reviews = reviews.merge(lanzamiento_item, on='reviews_item_id')
reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,posted_year,sentiment_analysis,release_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011,1,2009.0
1,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,2013,2,2013.0
2,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,2013,2,2013.0
3,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014,1,2012.0
4,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014,0,2013.0
...,...,...,...,...,...,...,...,...
52201,How51,http://steamcommunity.com/id/How51,440,No ratings yet,True,2014,1,2007.0
52202,76561198111410893,http://steamcommunity.com/profiles/76561198111...,304930,No ratings yet,True,2014,1,2017.0
52203,zaza147,http://steamcommunity.com/id/zaza147,265630,No ratings yet,True,2015,2,2014.0
52204,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015,1,2015.0


In [161]:
dfs = [reviews, gasto_items, ranking_genero, tiempo_juego_genero, items_developer]
# Nombres correspondientes a cada DataFrame
names = ['02-user_reviews', '04-gasto_items', '05-ranking_genero', '06-tiempo_juego_genero', '07-items_developer']

for df, name in zip(dfs, names):
    archivo = f'data/{name}_.csv'
    df.to_csv(archivo, index=False, encoding='utf-8')
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame '02-user_reviews' guardado como 'data/02-user_reviews_.csv'
DataFrame '04-gasto_items' guardado como 'data/04-gasto_items_.csv'
DataFrame '05-ranking_genero' guardado como 'data/05-ranking_genero_.csv'
DataFrame '06-tiempo_juego_genero' guardado como 'data/06-tiempo_juego_genero_.csv'
DataFrame '07-items_developer' guardado como 'data/07-items_developer_.csv'


Se pasan a parquet para optimizar el espacio para el deploy

In [162]:
for df, name in zip(dfs, names):
    archivo = f'data/{name}.parquet'
    pq.write_table(pa.Table.from_pandas(df), archivo)
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame '02-user_reviews' guardado como 'data/02-user_reviews.parquet'
DataFrame '04-gasto_items' guardado como 'data/04-gasto_items.parquet'
DataFrame '05-ranking_genero' guardado como 'data/05-ranking_genero.parquet'
DataFrame '06-tiempo_juego_genero' guardado como 'data/06-tiempo_juego_genero.parquet'
DataFrame '07-items_developer' guardado como 'data/07-items_developer.parquet'
